In [1]:
#1
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

#2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

#3
from sklearn.metrics import mean_absolute_error, accuracy_score

#4
from tensorflow.keras.layers import Input, Dense, Activation,Dropout

#5
from tensorflow.keras.models import Model

In [2]:
#6
from google.colab import files
import io

# Prompting to upload the train.csv
uploaded_train = files.upload()


Saving train.csv to train.csv


In [0]:
df = pd.read_csv(io.BytesIO(uploaded_train['train.csv']), index_col='PassengerId')

In [4]:
# Prompting to upload the test.csv file
uploaded_test = files.upload()

Saving test.csv to test.csv


In [0]:
# Save the testing data into a dataframe
df_test = pd.read_csv(io.BytesIO(uploaded_test['test.csv']), index_col='PassengerId')

In [10]:
#7
df.columns


Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
#Please check the test dataframe's columns
df_test.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

### Classify the features

In [12]:
#8
df.dtypes.unique

<bound method Series.unique of Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object>

In [0]:
#9
df['Family'] = df['SibSp'] + df['Parch']

In [0]:
#10
df = df.drop(columns=['SibSp', 'Parch'])

In [15]:
#11
df.isnull().sum()


Survived      0
Pclass        0
Name          0
Sex           0
Age         177
Ticket        0
Fare          0
Cabin       687
Embarked      2
Family        0
dtype: int64

In [0]:
#12
df = df.drop(columns=['Cabin'])

In [18]:
#13
df.groupby(['Pclass'])['Age'].mean()

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

In [0]:
#14
df['Age'] = df['Age'].fillna(df.groupby(['Pclass'])['Age'].transform(np.mean))

#df['Age'] = df['Age'].fillna(df.groupby(['Pclass'])['Age'].mean())

In [28]:
df['Age'].isnull().sum()

0

In [0]:
#15

df['Embarked'] = df['Embarked'].fillna('S')

In [30]:
df['Embarked'].isnull().sum()

0

In [57]:
#16
#df.isnull().sum()

df.isna().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
Ticket      0
Fare        0
Embarked    0
Family      0
dtype: int64

In [0]:
#17
df['Sex'].replace(['male','female'],[1,0],inplace=True)

In [59]:
df['Sex']

PassengerId
1      1
2      0
3      0
4      0
5      1
      ..
887    1
888    0
889    0
890    1
891    1
Name: Sex, Length: 891, dtype: int64

In [0]:
#18
X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Family']]

In [61]:
X

,Pclass,Sex,Age,Fare,Family
PassengerId,,,,,
1,3,1,22.00000,7.2500,1
2,1,0,38.00000,71.2833,1
3,3,0,26.00000,7.9250,0
4,1,0,35.00000,53.1000,1
5,3,1,35.00000,8.0500,0
...,...,...,...,...,...
887,2,1,27.00000,13.0000,0
888,1,0,19.00000,30.0000,0
889,3,0,25.14062,23.4500,3


In [0]:
#19
y = df[['Survived']]

In [63]:
y

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0
...,...
887,0
888,1
889,0


### Logistic Regression

In [0]:
#20
X_scaled = scale(X)

In [0]:
#21
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=101)

In [69]:
#22
print('X_train.shape: ', X_train.shape, 'y_train.shape: ', y_train.shape)

X_train.shape:  (712, 5) y_train.shape:  (712, 1)


### Neural Network

In [70]:
#23
input_layer = Input(shape=(X_train.shape[1]))
output = Dense(y_train.shape[1], activation='sigmoid')(input_layer)
model = Model(inputs=input_layer, outputs=output)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#24
from keras import layers, models
model.compile(optimizer='rmsprop', loss='mse', metrics=['acc'])

In [85]:
#25
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [86]:
#26
history = model.fit(X_train, y_train, batch_size=8, epochs=30, validation_split=0.2)

Train on 569 samples, validate on 143 samples
Epoch 1/30
569/569 [==============================] - 0s 255us/sample - loss: 0.1484 - acc: 0.7909 - val_loss: 0.1499 - val_acc: 0.7832
Epoch 2/30
569/569 [==============================] - 0s 169us/sample - loss: 0.1480 - acc: 0.7909 - val_loss: 0.1495 - val_acc: 0.7832
Epoch 3/30
569/569 [==============================] - 0s 164us/sample - loss: 0.1476 - acc: 0.7909 - val_loss: 0.1489 - val_acc: 0.7762
Epoch 4/30
569/569 [==============================] - 0s 177us/sample - loss: 0.1473 - acc: 0.7909 - val_loss: 0.1482 - val_acc: 0.7762
Epoch 5/30
569/569 [==============================] - 0s 158us/sample - loss: 0.1470 - acc: 0.7909 - val_loss: 0.1479 - val_acc: 0.7762
Epoch 6/30
569/569 [==============================] - 0s 163us/sample - loss: 0.1467 - acc: 0.7909 - val_loss: 0.1475 - val_acc: 0.7762
Epoch 7/30
569/569 [==============================] - 0s 188us/sample - loss: 0.1465 - acc: 0.7909 - val_loss: 0.1471 - val_acc: 0.7832
Ep

In [87]:
#27
score = model.evaluate(X_val, y_val)

179/179 [==============================] - 0s 91us/sample - loss: 0.1425 - acc: 0.7989


In [88]:
#28
print("test_score:", score[0])

test_score: 0.14251200904726316


In [92]:
#28
print("acc:", score[1])

acc: 0.79888266


In [0]:
#29
layer_1 = Dense(15, activation='relu')(input_layer)
layer_2 = Dense(10, activation='relu')(layer_1)

In [0]:
output = Dense(y_train.shape[1], activation='tanh')(layer_2)

In [0]:
neural_model = Model(inputs=input_layer, outputs=output)

In [0]:
#30
neural_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [100]:
#31
neural_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


In [101]:
#32
history = neural_model.fit(X_train, y_train, batch_size=8, epochs=30, validation_split=0.2)

Train on 569 samples, validate on 143 samples
Epoch 1/30
569/569 [==============================] - 0s 587us/sample - loss: 4.2366 - acc: 0.6257 - val_loss: 2.6319 - val_acc: 0.6713
Epoch 2/30
569/569 [==============================] - 0s 278us/sample - loss: 2.8100 - acc: 0.6432 - val_loss: 1.4560 - val_acc: 0.6853
Epoch 3/30
569/569 [==============================] - 0s 261us/sample - loss: 2.0538 - acc: 0.6749 - val_loss: 1.1026 - val_acc: 0.6853
Epoch 4/30
569/569 [==============================] - 0s 245us/sample - loss: 1.6271 - acc: 0.6784 - val_loss: 1.0386 - val_acc: 0.6993
Epoch 5/30
569/569 [==============================] - 0s 247us/sample - loss: 1.1372 - acc: 0.7030 - val_loss: 0.9391 - val_acc: 0.7203
Epoch 6/30
569/569 [==============================] - 0s 253us/sample - loss: 0.9084 - acc: 0.7399 - val_loss: 0.8312 - val_acc: 0.7273
Epoch 7/30
569/569 [==============================] - 0s 246us/sample - loss: 0.7475 - acc: 0.7522 - val_loss: 0.8228 - val_acc: 0.7622
Ep

In [102]:
#33
score = neural_model.evaluate(X_val, y_val)

179/179 [==============================] - 0s 61us/sample - loss: 0.4695 - acc: 0.8045


In [103]:
#34
print("test_score:", score[0], " and test_accuracy:", score[1])

test_score: 0.4695109560169987  and test_accuracy: 0.8044693


### Decision Tree

In [0]:
#35
decisiontree_model = DecisionTreeClassifier(random_state=101, max_leaf_nodes=50)

In [105]:
#36
decisiontree_model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=50,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=101, splitter='best')

In [0]:
#37
predict_tree = decisiontree_model.predict(X_val)

In [110]:
#38
print('acc: ', accuracy_score(y_val, predict_tree))

acc:  0.8156424581005587


### Preprocess, and fit test data

In [0]:
#39
df_test['Family'] = df_test['SibSp'] + df_test['Parch']

In [112]:
#40
df_test.groupby(['Pclass'])['Age'].mean()

Pclass
1    40.918367
2    28.777500
3    24.027945
Name: Age, dtype: float64

In [0]:
#41
df_test['Age'] = df_test['Age'].fillna(df_test.groupby(['Pclass'])['Age'].transform(np.mean))

In [0]:
#42
df_test['Embarked'] = df_test['Embarked'].fillna('S')

In [115]:
#43
sex_replace = {'female': 0, 'male': 1}
df_test = df_test.replace({'Sex': sex_replace})
df_test.astype({'Sex':'float32'}).dtypes

Pclass        int64
Name         object
Sex         float32
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
Family        int64
dtype: object

In [0]:
#44
scale_test = df_test[['Pclass', 'Sex', 'Age', 'Fare', 'Family']]
new_scaled = scale(scale_test)

In [0]:
#45
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

In [0]:

def accuracy_score(model):
  return np.mean(cross_val_score(model, x_train,y_train, cv=k_fold, scoring="accuracy"))


# Which approach did you choose and why?